In [ ]:
import azureCognitiveSearch
import vec2Text
import vec2TextSentenceTransformer

import pickle 


In [ ]:
# Load the cluster centers
with open(vec2Text.clusterCenterFile, 'rb') as pickle_in:
    clusterCenters = pickle.load(pickle_in)

In [ ]:
# Reset Index
azureCognitiveSearch.deleteIndex()
azureCognitiveSearch.createIndex()

In [ ]:
# Upload to Cognitive Search
max_batch_size = 100
documents = []

counter = 0
with open(vec2Text.dataFile) as f:
    for line in f:
        fields = line.split('\t')
        curVec = vec2TextSentenceTransformer.contentToMeanEmbedding(fields[1])
        vecText = ''
        for d in range(len(curVec)):
            vecText += vec2Text.convertFieldNumToString(d) + str(vec2Text.closest(clusterCenters[d], curVec[d])) + ' '

        documents.append({"@search.action": "upload","Id": vec2Text.stringToBase64(fields[0]), "Content": fields[1],"VecText": vecText })

        counter += 1
        if len(documents) == max_batch_size:
            azureCognitiveSearch.uploadDocuments(documents)
            documents = []
            print ('Processed:', counter)

if len(documents) > 0:
    azureCognitiveSearch.uploadDocuments(documents)
    documents = []
    print ('Processed:', counter)
